In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np, pandas as pd, sklearn
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression as logistic_reg
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier as knn_model
from sklearn.svm import SVC

In [2]:
'''
try models with actual age value without col for nan values
use col for nans in other cols besides age
'''

'\ntry models with actual age value without col for nan values\nuse col for nans in other cols besides age\n'

In [3]:
def scale(dfs, *args):
    for df in dfs:
        for col in args: 
            df[col] = preprocessing.MinMaxScaler().fit_transform(np.array(df[col]).reshape(-1, 1))

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# integer division by 10 to make a label between 1-8 for age groups, if the number is a NaN, then just replace with a median
age_group = lambda df, m=10: ['a' + str(int(a / m)) if a == a else 'a2' for a in df['age']]

for df in (train, test):
    df.columns = [a.lower() for a in df.columns]
   
    df['cabin'] = [int(a[0].lower(), base=36) - 10 if type(a) is str else -1 for a in df['cabin']]
    df['sex'] = [1 if a=='male' else -1 for a in df['sex']]
    
    df.index = df['passengerid']
    df = df.drop('passengerid', axis=1)
    
y = train['survived']
# train = train.drop('survived', axis=1)
score_model = lambda model, x: accuracy_score(model.predict(x), y)

scatter = lambda cols: pd.plotting.scatter_matrix(train[cols], alpha=0.2, figsize=(10,5));
plotter = lambda col: plt.hist(train[col])

numbers_cols = ['age', 'sex', 'pclass', 'sibsp', 'parch', 'fare', 'cabin']
numbers_train = train[numbers_cols]
numbers_test = test[numbers_cols]

numbers_train['age'].fillna((numbers_train['age'].mean()), inplace=True)
numbers_test['age'].fillna((numbers_test['age'].mean()), inplace=True)

def new_col(df, df2, col, name, f): 
    df2[name] = f(df[col])
    return df2

new_col(train, numbers_train, 'age', 'agenan', lambda l: [a!=a for a in l])
new_col(test, numbers_test, 'age', 'agenan', lambda l: [a!=a for a in l])

train['age'] = age_group(train)
test['age'] = age_group(test)

numbers_test['fare'][1044] = 0.015282
#scale((numbers_train, numbers_test), 'fare')

s = lambda a: a.split(',')[1].split('.')[0].strip()
titles = list(zip(*[('Master', 21), ('Mrs', 72), ('Miss', 78), ('Mr', 240)]))[0]
s2 = lambda l: [s(a) if s(a) in titles else 'other' for a in l]
train['title'] = s2(train['name'])
test['title'] = s2(test['name'])

dummies = lambda df, df2, col: df.join(pd.get_dummies(df2[col]))
numbers_train = dummies(numbers_train, train, 'age')
numbers_test = dummies(numbers_test, test, 'age')

numbers_train = dummies(numbers_train, train, 'title')
numbers_test = dummies(numbers_test, test, 'title')

numbers_train = numbers_train.drop('a8', axis=1)

In [5]:
l = [(a.isnumeric(), y.iloc[i]) for i, a in enumerate(train['ticket'])]
sum(a[1] for a in l if a[0])/sum(list(zip(*l))[0]), sum(a[1] for a in l if not a[0])/(len(l) - sum(list(zip(*l))[0]))

(0.3842662632375189, 0.3826086956521739)

In [6]:
def gridsearch(model, params):
    asda = GridSearchCV(model, params)
    asda.fit(numbers_train, y)
    return asda.best_params_, asda

In [7]:
svm = SVC().fit(numbers_train, y)
score_model(svm, numbers_train)

0.6891133557800224

In [8]:
knn = knn_model(**{'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}).fit(numbers_train, y)
score_model(knn, numbers_train)

0.8709315375982043

In [9]:
most_frequent = DummyClassifier(strategy='most_frequent').fit(train, y)

log_reg = logistic_reg().fit(numbers_train, y)
score_model(log_reg, numbers_train)

0.8305274971941639

In [23]:
forest = 
score_model(forest, numbers_train)

0.8866442199775533

In [37]:
params = {'criterion': 'entropy',
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'n_estimators': 90}

afas = lambda b: [1 if sum(a)/b >= 0.5 else 0 for a in zip(*[rf(**params).fit(numbers_train, y).predict(numbers_test) for i in range(b)])]

# SCORES:
<b> Dummy (most frequent): </b>0.61  
<b> Log Reg:</b> 0.714  
<b> Log Reg (scaled):</b> 0.717  
<b> Log Reg (with dummies) </b> 0.718  
<b> Log Reg (peak) </b> 0.821  
<b> Random Forest </b> 0.88    
pos: 11144 => 8906 (0.76794) => 0.77511 => 0.76794 => 0.77033 => 0.67703

In [18]:
def save(model):
    pd.DataFrame(columns=['PassengerId', 'Survived'], data=zip(*[model.predict(numbers_test), numbers_test.index][::-1])).to_csv('final_results.csv', index=False)
    pd.read_csv('final_results2.csv')

In [43]:
pd.DataFrame(columns=['PassengerId', 'Survived'], data=zip(numbers_test.index, afas(5))).to_csv('final_results2.csv', index=False)